In [12]:
!pip install nltk

In [13]:
import pandas as pd
import numpy as np
import json
import ast
from nltk.tokenize import WhitespaceTokenizer
import random

In [73]:
# Retrieving the different Dataset stored in csv files

usernames_df = pd.read_csv("../mitchall.boesel/Twitter/tweets.csv", names=["Data", "Label"], header=0)
hostnames_df = pd.read_csv("../caesar.white/hostnames_strings.csv", names=["Data", "Label"], header=0)
filenames_df = pd.read_csv("../jongyun.kim/featuriaztion/filenames.csv", names=["Data", "Labels"], header=0)

In [74]:
# splitting the datasets into positive and negative lists
def IsPositive(r):
    return True if r[1] else False
def IsNegative(r):
    return True if not r[1] else False

username_list = usernames_df.values.tolist()
hostnames_list = hostnames_df.values.tolist()
filenames_list = filenames_df.values.tolist()

positive_dataset = list(filter(IsPositive, username_list+hostnames_list+filenames_list))
negative_dataset = list(filter(IsNegative, username_list+hostnames_list+filenames_list))

random.shuffle(positive_dataset)
random.shuffle(negative_dataset)

print(len(positive_dataset))
print(positive_dataset[:10])
print(len(negative_dataset))
print(negative_dataset[:10])


25839
[['/reset.css', 1], ['RT SamsungMobile:  Boom! BTS_twt - We couldnt be more proud of your #GRAMMYs nomination! You guys are dynamite!  #GrammyNominatedB', 1], ['RT shenna_brook: being loved like a morgan wallen song &gt;', 1], ['/blog/geekery/puppet-nodeless-configuration..cs', 1], ['/projects/firefox-urledit..java', 1], ['/presentations/logstash-scale11x/plugin/zoom-js/zoom.js', 1], ['/reset.css', 1], ['/images/web/2009/banner.png', 1], ['RT OnActuate: How do you prioritise datacenter discovery to accelerate cloud migration? https://t.co/SElM7nALny #microsoft #onactuate #ms', 1], ['RT chrysallium: YEAHHHH CONGRATSSS Ohmmmmmmyyyyyyyyggghghhhhhhhgggggggggdhdhsjsixudbslsogbdsisgshdbxidjdbdidhdifjfiri #GRAMMYs #BTS #Gra', 1]]
10137
[['159232232246', 0], ['A seamless phone-to-PC experience, right in your hands. Get more out of your #GalaxyNote20 apps with Microsoft. https://t.co/8Mhtld366g', 0], ['Office 365 Roadmap | Microsoft Information Protection: Data-at-Rest Encryption for Micr

In [75]:
# Combining the dataset into 1
dataset = []
l = len(negative_dataset) if len(negative_dataset) < len(positive_dataset) else len(positive_dataset)

for i in range(0,l):
    dataset.append(positive_dataset[i])
    dataset.append(negative_dataset[i])
print(dataset[:5])
print(len(dataset))

[['/reset.css', 1], ['159232232246', 0], ['RT SamsungMobile:  Boom! BTS_twt - We couldnt be more proud of your #GRAMMYs nomination! You guys are dynamite!  #GrammyNominatedB', 1], ['A seamless phone-to-PC experience, right in your hands. Get more out of your #GalaxyNote20 apps with Microsoft. https://t.co/8Mhtld366g', 0], ['RT shenna_brook: being loved like a morgan wallen song &gt;', 1]]
20274


In [39]:
# Features

# Username specific features
def LowerUnderscoreUpper(s):
    l = len(s)

    for i in range(0,l-2):
        if s[i].islower() and s[i+1] == '_' and s[i+2].isupper():
            return True
    return False

def HasUnderscore(s):
    s = str(s)

    for c in s:
        if c == '_':
            return True
    return False

def LowerUpperLower(s):
    s = str(s)

    for i in range(0, len(s)-2):
        if s[i].islower() and s[i+1].isupper() and s[i+2].islower():
            return True
    return False

def MultipleLowerUpperLower(s):
    s = str(s)

    flag = False
    for i in range(0, len(s)-2):
        if s[i].islower() and s[i+1].isupper() and s[i+2].islower():
            if flag:
                return True
            else:
                flag = True
    return False

def ExactlyTwoUppercase(s):
    count = 0

    for c in s:
        if c.isupper():
            count+=1
    if count ==2:
        return True
    else:
        return False

def AllLowerMoreThan(bound, s):
    s = str(s)

    if len(s) < bound:
        return False
    for c in s:
        if c.isupper():
            return False
    return True

def AdjacentUppers(s):
    s = str(s)

    for i in range(0,len(s)-1):
        if s[i].isupper() and s[i+1].isupper():
            return True
    return False

def StartLetterEndNonLetter(s):
    s = str(s)

    if s[0].isalpha() and not s[-1].isalpha():
        return True
    return False


# Hostname specific features
def LengthGTLT(gt, ls, s):
    l = len(str(s))
    if l >= gt and l <= ls:
        return True
    else:
        return False

def IllegalHostnameChars(s):
    s = str(s)
    illegal_chars = [".", "\", ""/", "*", "?", "\"", "<", ">", "|", ",", "~", ":", "!", "@", "#", "$", "%", "^", "&", "'", "(", ")", "{", "}", " "]
    for c in s:
        if c in illegal_chars:
            return True
    return False

def AlphaOrDigit(s):
    s = str(s)
    if s.isalpha() or s.isdigit():
        return True
    return False

def HostIllegalEnding(s):
    s = str(s)
    if s[-1] == '-' or s[-1] == '.':
        return True
    return False

# Filename specific features

def ContainsPeriod(s):
    s = str(s)
    for c in s:
        if c == '.':
            return True
    return False

def IsUrl(s):
    s = str(s)
    l = len(s)
    if l > 3:
        if s[0:4] == "http":
            return True
    if l > 4:
        if s[0:5] == "https":
            return True
    return False

def HasSlash(s):
    s = str(s)
    for c in s:
        if c == '/' or c == '\\':
            return True
    return False

def HasMultipleSlash(s):
    s = str(s)
    slashes = ['/','\\']
    flag = False
    for c in s:
        if not flag and c in slashes:
            flag = True
        elif flag and c in slashes:
            return True
    return False

def HasPossibleExtension(s):
    s = str(s)
    for i in range(0,len(s)):
        if s[i] == '.':
            x = s[i:-1]
            if len(x) >= 2 and len(x) <= 4:
                return True
    return False


In [76]:
# extracting features and labels
feature_list = []
labels = []
 
tk = WhitespaceTokenizer() 

for dp in dataset:
    d = dp[0]
    label = dp[1]
    tokens = tk.tokenize(d)
    tokens = list(filter(lambda x: not IsUrl(x), tokens))
    dp_list = []
    dp_list.append(True) if True in list(map(lambda x: LowerUnderscoreUpper(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasUnderscore(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: LowerUpperLower(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: MultipleLowerUpperLower(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: ExactlyTwoUppercase(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AllLowerMoreThan(10,x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AdjacentUppers(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: StartLetterEndNonLetter(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: LengthGTLT(1, 15, x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: IllegalHostnameChars(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: AlphaOrDigit(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HostIllegalEnding(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: ContainsPeriod(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasMultipleSlash(x), tokens)) else dp_list.append(False)
    dp_list.append(True) if True in list(map(lambda x: HasPossibleExtension(x), tokens)) else dp_list.append(False)
    feature_list.append(dp_list)
    labels.append(label)


In [61]:
# BEGIN MACHINE LEARNING MODELS
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [64]:
def calcAccuracy(target, predictions):
    total = len(predictions)
    correct = 0
    for i in range(len(predictions)):
        if target[i] == predictions[i]:
            correct += 1
        accuracy = float(correct) / float(total)
    return accuracy

def FalsePositives(x_test, y_test, pred):
    false_positives = []
    for i in range(0, len(x_test)):
        if y_test[i] == 0 and pred[i] == 1:
            false_positives.append(x_test[i])
    return false_positives

def FalseNegatives(x_test, y_test, pred):
    false_negatives = []
    for i in range(0,len(x_test)):
        if y_test[i] == 1 and pred[i] == 0:
            false_negatives.append(x_test[i])
    return false_negatives

def PrintResults(classifier, acc, recall, precision):
    print(classifier + " Classifier")
    print("Accuracy= ", acc)
    print("Precision= ", precision)
    print("Recall= ", recall)

In [77]:
X = feature_list # all the features
y = labels     # labels, 1 or 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
print(len(X))

20274


In [78]:
# DECISION TREE CLASSIFIER
dt = DecisionTreeClassifier().fit(X_train, y_train)
pred = dt.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Decision Tree", acc, recall, precision)

false_negatives = FalseNegatives(X_test, y_test, pred)
false_positives = FalsePositives(X_test, y_test, pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)}")

Decision Tree Classifier
Accuracy=  0.92063966522194
Precision=  0.9199160167966407
Recall=  0.9207445211648154

FP-> 267 
FN-> 264


In [79]:
# KNN CLASSIFIER
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("KNN", acc, recall, precision)

false_negatives = FalseNegatives(X_test, y_test, pred)
false_positives = FalsePositives(X_test, y_test, pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)}")

KNN Classifier
Accuracy=  0.9073382155133761
Precision=  0.8932404989846243
Recall=  0.924347042930051

FP-> 368 
FN-> 252


In [80]:
# LOGISTIC REGRESSION CLASSIFER
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Logistic Regression", acc, recall, precision)


Logistic Regression Classifier
Accuracy=  0.8756538633985951
Precision=  0.8950363578880809
Recall=  0.8498949264485139


In [81]:
# RANDOM FORREST
rfm = RandomForestClassifier().fit(X_train, y_train)
pred = rfm.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Random Forrest", acc, recall, precision)

false_negatives = FalseNegatives(X_test, y_test, pred)
false_positives = FalsePositives(X_test, y_test, pred)

print(f"\nFP-> {len(false_positives)} \nFN-> {len(false_negatives)}")

Random Forrest Classifier
Accuracy=  0.923927664026304
Precision=  0.9219497607655502
Recall=  0.925547883518463

FP-> 261 
FN-> 248


In [82]:
# NAIVE BAYS
nb = GaussianNB().fit(X_train, y_train)
pred = nb.predict(X_test)
acc = calcAccuracy(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
PrintResults("Naive Bays", acc, recall, precision)

Naive Bays Classifier
Accuracy=  0.7188761022268719
Precision=  0.838468720821662
Recall=  0.5391774241969378


In [58]:
# Try/Catch for recognizing if input is json or not

#data_dict = list(map(lambda x : ast.literal_eval(x), data))
for x in dataset[:5]:
    print(x)

for y in dataset:
    x = None
    try:
        x = ast.literal_eval(y[0])
        #print(f"It didn't break! {x}")
    except (ValueError, SyntaxError):
        z = True
        #print(f"It broke:( {y}") 



['#ReTweetThis for #OurMischief #BBlogRT SGH_RTs GFXCoach FearRTs ShoutGamers Demented_RTs Mighty_RTs DynoRTs NightRetweets WitWGARA posted: https://t.co/oAcPcjMyvAWatch Ninja now streaming VALORANTat https://t.co/QghgQSwvrL- #ThankYou for being #OurMischief! Wit', 1]
['TRUDYDELEON-', 0]
['groucho-eu', 1]
['7JERRY-NELSON', 0]
['RT tkasasagi: My Kuzushiji recognition mobile app using Tensorflow and Flutter project got awarded research grant from Japan Science and T', 1]


In [59]:
tk = WhitespaceTokenizer() 
t1 = "Hello"
token = tk.tokenize(t1)
print(token)

['Hello']
